In [ ]:
from pathlib import Path
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import scipy.io
import matplotlib.pyplot as plt
data_folder=Path('')

In [ ]:
Y = pd.read_csv(data_folder / 'REFERENCE.csv')

In [ ]:
def process_data(data_entry):
    signal = torch.tensor(data_entry['ECG'][0][0][2][:,:15000], dtype=torch.float32)
    if signal.shape[1] < 15000:
        n_pad = 15000 - signal.shape[1]
        signal = F.pad(signal, (0, n_pad), mode='constant')
    age = torch.tensor(data_entry['ECG'][0][0][1][0, 0], dtype=torch.float32)
    if data_entry['ECG'][0][0][0][0]==np.str_('Male'):sex=torch.tensor(1,dtype=torch.float32)
    else:sex=torch.tensor(0,dtype=torch.float32)
    return signal.unsqueeze(0), age.unsqueeze(0), sex.unsqueeze(0)

def load_raw_data_icbeb(df):
    data = [scipy.io.loadmat('/remote-home/zhuyl/CPSC2018/CPSC/' + str(f)) for f in tqdm(df.Recording)]
    signals = []
    ages = []
    sexes = []
    for data_entry in tqdm(data):
        signal, age, sex = process_data(data_entry)
        signals.append(signal)
        ages.append(age)
        sexes.append(sex)
    signals = torch.cat(signals, dim=0)
    ages = torch.cat(ages, dim=0)
    sexes = torch.cat(sexes, dim=0)
    return signals, ages, sexes


In [ ]:
datas, ages, sexes=load_raw_data_icbeb(Y)

In [ ]:
y=torch.tensor(np.array([i for i in tqdm(Y.First_label)]))
y=y-1
num_classes = int(y.max()+1)
one_hot_tensor = torch.nn.functional.one_hot(y, num_classes)